![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

# Algoritmo de Clasificación II

## Regresión Logística

La regresión logística es un tipo de modelo de aprendizaje automático supervisado que se utiliza para predecir la probabilidad de que un evento ocurra. La regreión logística se basa en la función logística, que es una función sigmoide que toma un valor entre 0 y 1. Aunque se llama *regresión*, se utiliza comúnmente para problemas de clasificación binaria (dos clases), aunque también se puede utilizar para clasificación multiclase.

La regresión logística se puede utilizar en los siguientes casos:

- **Cuando la variable dependiente es categórica**. La regresión logística se utiliza para predecir variables dependientes que pueden tomar un número limitado de valores.
- **Probabilidad de eventos**. Cuando se desea modelar la probabilidad de un evento en función de variables predictoras.

![Regresión logística](https://d1.awsstatic.com/S-curve.36de3c694cafe97ef4e391ed26a5cb0b357f6316.png)

#### Ventajas y desventajas

**Ventajas**

1. **Interpretación probabilística**: Proporciona probabilidades directas de pertenencia a una clase, lo que facilita la interpretación.
2. **Eficiente para clasificación binaria**.
3. **No requiere linealidad**: No asume linealidad en la relación entre las variables predictoras y la probabilidad de pertenencia a una categoría.

**Desventajas**

1. **Sensibilidad a características irrelevantes**.
2. **No es adecuada para modelar relaciones no lineales complejas**.

Notas mias: solo funciona para problemas binarios.   Si la probabilidad es mayor a 0,5 si, menor a 0,5 false.

In [1]:
# Cargamos las librerias
import pandas as pd
import numpy as np

In [2]:
# google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# cargamos los datos
data = pd.read_csv('/content/drive/MyDrive/samples/Social_Network_Ads.csv')


In [4]:
data.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [6]:
x = data.iloc[:,[2,3]]

In [ ]:
x

In [10]:
x.columns

Index(['Age', 'EstimatedSalary'], dtype='object')

In [ ]:
y = data.iloc[:, -1].values
y

In [13]:
gender = data[["Gender"]]

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
cat_encoder = OneHotEncoder()

In [16]:
data_cat_1hot = cat_encoder.fit_transform(gender)

In [17]:
cat_encoder.categories_

[array(['Female', 'Male'], dtype=object)]

In [18]:
cat_encoder.get_feature_names_out()

array(['Gender_Female', 'Gender_Male'], dtype=object)

In [19]:
data_cat_1hot.toarray()[:3]

array([[0., 1.],
       [0., 1.],
       [1., 0.]])

In [20]:
encoded_df = pd.DataFrame(data_cat_1hot.toarray(), columns = cat_encoder.get_feature_names_out())
encoded_df.head()

,Gender_Female,Gender_Male
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [21]:
data1 = pd.concat([x, encoded_df], axis = 1)
data1.head()

,Age,EstimatedSalary,Gender_Female,Gender_Male
0,19,19000,0.0,1.0
1,35,20000,0.0,1.0
2,26,43000,1.0,0.0
3,27,57000,1.0,0.0
4,19,76000,0.0,1.0


In [22]:
data1.shape

(400, 4)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [25]:
x_train.shape

(300, 2)

In [26]:
x_test.shape

(100, 2)

Dado que tenemos salarios valores bastantes altos y las edades son bastantes bajas comparadas con los salarios vamos a hacer un escalado de los datos

El escalado de datos en Machine Learning es una técnica fundamental para mejorar el rendimiento de los modelos, especialmente en algoritmos que dependen de la distancia (como Regresión Logística, SVM, KNN y Redes Neuronales).

Escalado

In [27]:
# importamos las librerias
from sklearn.preprocessing import StandardScaler

In [30]:
sc_x = StandardScaler()

In [31]:
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

Ahora si vamos a entrenar nuestro algoritmo de regresion logistica

In [32]:
# importamos las librerias y entrenamos
from sklearn.linear_model import LogisticRegression

In [33]:
# creamos una instancia y entrenamos el modelos
log_reg = LogisticRegression(random_state = 0)
log_reg.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [34]:
y_predic = log_reg.predict(x_test)

In [37]:
print(f"Reales: {y_test[:10]}")
print(f"Predicciones: {y_predic[:10]}")

Reales: [0 0 0 0 0 0 0 1 0 0]
Predicciones: [0 0 0 0 0 0 0 1 0 1]


Matriz de confusion para medir el rendimiento

|||
|--|--|
|TN|FP|
|FN|TP|

TN: predice negativo y es negativo <br>
FP: Predice positivo cuando en realidad es negativo <br>
FN: Predice negativo cuando en realidad es positivo <br>
TP:  predice positivo y es positivo <br>


In [38]:
# calculamos la matriz de confusion
from sklearn.metrics import confusion_matrix

In [39]:
confusion_matrix(y_test, y_predic)

array([[65,  3],
       [ 8, 24]])

vemos los parametros de presicion y memoria

In [40]:
from sklearn.metrics import precision_score, recall_score

In [41]:
precision_score(y_test, y_predic)

0.8888888888888888

In [43]:
recall_score(y_test, y_predic)

0.75